Video Game Reviews Text Analysis

By Fei Guo, Victor Li, Jonathan Lin, Xinyu Zheng

INFX 575 Calulate Jargon Distance of similar authors

In [1]:
# read from results.csv file
import csv
        
reader = csv.reader(open("results.csv", 'rU'), dialect='excel')

dates = []
games = []
links = []
platforms = []
reviews = []
writers = []
scores = []
scores_100 = []
sites = []

for col in reader:
    dates.append(col[0])
    games.append(col[1])
    links.append(col[2])
    platforms.append(col[3])
    reviews.append(col[4])
    writers.append(col[5])
    scores.append(col[6])
    scores_100.append(col[7])
    sites.append(col[8])
    
dates = dates[1:]
games = games[1:]
links = links[1:]
platforms = platforms[1:]
reviews = reviews[1:]
writers = writers[1:]
scores = scores[1:]
scores_100 = scores_100[1:]
sites = sites[1:]


In [2]:
# save to a data frame

import pandas as pd
import numpy as np

index = range(len(games))
columns = ['dates','games','links','platforms','reviews','writers','scores','scores_100','sites']

review_data = pd.DataFrame(index=index, columns=columns)
review_data['dates'] = dates
review_data['games'] = games
review_data['links'] = links
review_data['platforms'] = platforms
review_data['reviews'] = reviews
review_data['writers'] = writers
review_data['scores'] = scores
review_data['scores_100'] = scores_100
review_data['sites'] = sites

review_data.ix[:1, :9]

,dates,games,links,platforms,reviews,writers,scores,scores_100,sites
0,2008-05-23 13:00:00,SingStar,http://www.giantbomb.com/api/review/1900-1/,PlayStation 3,All these karaoke video games have got it all ...,Jeff Gerstmann,4.0,80.0,Giant Bomb
1,2008-05-20 17:00:00,Haze,http://www.giantbomb.com/api/review/1900-2/,PlayStation 3,Haze attempts to deal with the concept of wart...,Jeff Gerstmann,2.0,40.0,Giant Bomb


In [3]:
# create a new data frame and populate with writers, combined reviews of each writer, and average scores of each writer

import pandas as pd
import numpy as np

index = range(len(writers))
columns = ['writers','scores','reviews']

author_review = pd.DataFrame(index=index, columns=columns)
author_review['writers'] = writers
author_review['scores'] = scores_100
author_review['scores'] = author_review['scores'].convert_objects(convert_numeric=True)
author_review['reviews'] = reviews

# print author_review.ix[:1, :3]
# print author_review.shape

author_review['reviews'] = author_review[['writers','scores','reviews']].groupby(['writers'])['reviews'].transform(lambda x: ','.join(x))
# author_review = author_review[['writers','scores','reviews']].drop_duplicates()
author_review = author_review.reset_index()
# print author_review

author_avg_scores = author_review.groupby(['writers','reviews'], as_index=False)
author_avg_scores = author_avg_scores['scores'].mean()
author_avg_scores = pd.DataFrame(author_avg_scores)
# print author_avg_scores

In [5]:
# calculate average score of all writers
review_data['scores_100'] = review_data['scores_100'].convert_objects(convert_numeric=True)
avg_score = float(sum(review_data['scores_100']))/len(review_data['scores_100'])
print avg_score

70.7648648649


In [32]:
sorted_scores = author_avg_scores.sort(['scores'], ascending=True)

sorted_scores = sorted_scores.reset_index()
sorted_scores

groups_writers = [[]]*10
groups_reviews = [[]]*10
groups_scores = [[]]*10

for k in range(10):
    groups_writers[k] = []
    groups_reviews[k] = []
    groups_scores[k] = []

for i in range(len(sorted_scores)):   
    current_writer = sorted_scores.ix[i,'writers']
    current_review = sorted_scores.ix[i,'reviews']
    current_score = sorted_scores.ix[i,'scores']
    
    for j in range(10):       
        if (current_score <= (j+1)*10) and (current_score > j*10):
            groups_writers[j].append(current_writer)
            groups_reviews[j].append(current_review)
            groups_scores[j].append(current_score)


In [53]:
# ignore this section

import pandas as pd
import numpy as np

df = [[]]*10

for i in range(10):
    index = range(len(groups_writers[i]))
    column = ['writers','reviews','scores']
    df[i] = pd.DataFrame(index=index, columns=column)
    df[i]['writers'] = groups_writers[i]
    df[i]['reviews'] = groups_reviews[i]
    df[i]['scores'] = groups_scores[i]


In [64]:

def clean_words(token_words,index):
    uni_cleaned = ''
    vocab_tmp = []
    stemmer=stem.PorterStemmer()
    for w in token_words:
        w = w.lower()
        if w not in stopset:
            tmp = stemmer.stem(w)
            tmp = unicodedata.normalize('NFKD', tmp).encode('ascii','ignore')
            if tmp not in string.punctuation:
                if not tmp.isdigit():
                    if tmp not in letters:
                        if tmp not in special_char:                               
                            vocab_tmp.append(tmp)
    for u in vocab_tmp:
        if (vocab_tmp.count(u) >= 5):
            uni_cleaned = uni_cleaned + u + ' '
            vocab[index].append(u)
    return uni_cleaned

In [65]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import unicodedata
from nltk import stem
import string

stopset = set(stopwords.words('english'))
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
special_char = ['~~','``',"''", '--',"'ll","n't","'re","'s","'d"]

vocab = [[]]*10
for i in range(10):
    vocab[i] = []

for i in range(len(groups_writers[0])):
    with open('/Users/XinyuZ/group_1/'+groups_writers[0][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[0][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,0)
        f.write(cleaned_review)
        f.close()

for i in range(len(groups_writers[1])):
    with open('/Users/XinyuZ/group_2/'+groups_writers[1][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[1][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,1)
        f.write(cleaned_review)
        f.close()

for i in range(len(groups_writers[2])):
    with open('/Users/XinyuZ/group_3/'+groups_writers[2][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[2][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,2)
        f.write(cleaned_review)
        f.close()

for i in range(len(groups_writers[3])):
    with open('/Users/XinyuZ/group_4/'+groups_writers[3][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[3][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,3)
        f.write(cleaned_review)
        f.close()
        
for i in range(len(groups_writers[4])):
    with open('/Users/XinyuZ/group_5/'+groups_writers[4][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[4][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,4)
        f.write(cleaned_review)
        f.close()
        
for i in range(len(groups_writers[5])):
    with open('/Users/XinyuZ/group_6/'+groups_writers[5][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[5][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,5)
        f.write(cleaned_review)
        f.close()
        
for i in range(len(groups_writers[6])):
    with open('/Users/XinyuZ/group_7/'+groups_writers[6][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[6][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,6)
        f.write(cleaned_review)
        f.close()
        
for i in range(len(groups_writers[7])):
    with open('/Users/XinyuZ/group_8/'+groups_writers[7][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[7][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,7)
        f.write(cleaned_review)
        f.close()
        
for i in range(len(groups_writers[8])):
    with open('/Users/XinyuZ/group_9/'+groups_writers[8][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[8][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,8)
        f.write(cleaned_review)
        f.close()
        
for i in range(len(groups_writers[9])):
    with open('/Users/XinyuZ/group_10/'+groups_writers[9][i]+'.txt', 'w') as f:
        tokens=word_tokenize(str(groups_reviews[9][i]).decode("ISO-8859-1"))
        cleaned_review = clean_words(tokens,9)
        f.write(cleaned_review)
        f.close()

In [68]:
group_vocab = [[]]*7

group_vocab[0] = vocab[1]
group_vocab[1] = vocab[4]
group_vocab[2] = vocab[5]
group_vocab[3] = vocab[6]
group_vocab[4] = vocab[7]
group_vocab[5] = vocab[8]
group_vocab[6] = vocab[9]


In [70]:
# generate corpus with 'cleaned' groups
corpus_vocab = [[]]*7

for i in range(0,6):
    corpus_vocab[i] = [[]]*7
    for j in range(i+1,7): 
        corpus_vocab[i][j] = []
        for wi in group_vocab[i]:
            corpus_vocab[i][j].append(wi)
        for wj in group_vocab[j]:
            corpus_vocab[i][j].append(wj)

In [72]:
import math
from collections import Counter

# calculate H value for each group
def get_H(group_word):
    log_sum_group = 0
    prob_count = Counter(group_word)
    freq_sum = sum(prob_count.itervalues())
    vocab = list(set(group_word))
    for w in vocab:  
        w_prob = prob_count[w]/float(freq_sum)
        if (w_prob > 0):
            log_sum_group += -w_prob * math.log(w_prob,2)
    return log_sum_group

H = [0]*7
for i in range(0,7):
    H[i] = get_H(group_vocab[i])
    print H[i]

4.42458642082
6.26341431311
9.36071008837
10.3726120367
10.1739253829
8.94798903083
6.43430583605


In [73]:
# THIS SECTION OF CODE TOOK ABOUT 40 MINUTES TO FINISH RUNNING AND I CANNOT FIGURE OUT HOW TO OPTIMIZE
import math
from collections import Counter

# calculate Q values between groups
def get_Qij(groupi_word,groupj_word,groupij_word):
    tmp_Qij = 0
    alpha = 0.01
    
    count = Counter(groupi_word)
    freq = sum(count.itervalues())
    vocab = list(set(groupi_word))
    
    count_ij = Counter(groupij_word)
    freq_ij = sum(count_ij.itervalues())
    
    count_j = Counter(groupj_word)
    freq_j = sum(count_j.itervalues())
    
    for w in vocab:
        pi = (1-alpha)*count[w]/float(freq)+alpha*count_ij[w]/float(freq_ij)
        pj = (1-alpha)*count_j[w]/float(freq_j)+alpha*count_ij[w]/float(freq_ij)

        tmp_Qij += pi * math.log(pj,2)
    Q_ij = 0 - tmp_Qij
    return Q_ij

Q = [[]]*7
for i in range(0,7):
    Q[i] = [[]]*7

for i in range(0,6):   
    for j in range(i+1,7):
        Q[i][j] = get_Qij(group_vocab[i],group_vocab[j],corpus_vocab[i][j])
        Q[j][i] = get_Qij(group_vocab[j],group_vocab[i],corpus_vocab[i][j])


In [74]:
# calculate E values between groups
E = [[]]*7
for i in range(0,7):
    E[i] = [[]]*7
    
for i in range(0,6):   
    for j in range(i+1,7):
        E[i][j] = H[i] / Q[i][j]
        E[j][i] = H[j] / Q[j][i]

In [75]:
# calculate C values between groups
C = [[]]*7
E_C = [[]]*7
for i in range(0,7):
    C[i] = [[]]*7
    E_C[i] = [[]]*7
for i in range(0,6):   
    for j in range(i+1,7):
        C[i][j] = 1 - E[i][j]
        C[j][i] = 1 - E[j][i]
        E_C[i][j] = (C[i][j]+C[j][i])/2
        E_C[j][i] = (C[i][j]+C[j][i])/2

for i in range(0,7):
    E_C[i][i] = 0

In [77]:
# calculate average C values for each group
C_avg = [0]*7

for i in range(0,7):
    for j in range(0,7):
        if (i != j):
            C_avg[i] += C[i][j]
    C_avg[i] = C_avg[i]/6
    print C_avg[i]

0.603877311999
0.354558169854
0.232642618479
0.222251450889
0.218852374037
0.243247576346
0.389665200183
